In [ ]:
import pandas as pd

# Cargo el archivo CSV
df_consum_2022 = pd.read_csv('2022_consum_bcn.csv', header=0, delimiter= ",")

df_consum_2022.head(10)

,Any,Data,Codi_Postal,Sector_Economic,Tram_Horari,Valor
0,2022,2022-01-01,8001,Indústria,De 00:00:00 a 05:59:59 h,817
1,2022,2022-01-01,8001,Indústria,De 06:00:00 a 11:59:59 h,942
2,2022,2022-01-01,8001,Indústria,De 12:00:00 a 17:59:59 h,1145
3,2022,2022-01-01,8001,Indústria,De 18:00:00 a 23:59:59 h,1184
4,2022,2022-01-01,8001,Indústria,No consta,0
5,2022,2022-01-01,8001,Residencial,De 00:00:00 a 05:59:59 h,24464
6,2022,2022-01-01,8001,Residencial,De 06:00:00 a 11:59:59 h,24860
7,2022,2022-01-01,8001,Residencial,De 12:00:00 a 17:59:59 h,36160
8,2022,2022-01-01,8001,Residencial,De 18:00:00 a 23:59:59 h,38349
9,2022,2022-01-01,8001,Residencial,No consta,0


La composición del dataframe consta de 251070 filas y 6 columnas

In [3]:
df_consum_2022.shape

(251070, 6)

Reviso el tipo de datos que tienen en este momento para determinar si hay que realizar algunas modificaciones

In [4]:
df_consum_2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251070 entries, 0 to 251069
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   Any              251070 non-null  int64 
 1   Data             251070 non-null  object
 2   Codi_Postal      251070 non-null  int64 
 3   Sector_Economic  251070 non-null  object
 4   Tram_Horari      251070 non-null  object
 5   Valor            251070 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 11.5+ MB


Confirmo la presencia de valores nulos (no se encuentran)

In [5]:
df_consum_2022.isnull().sum()

Any                0
Data               0
Codi_Postal        0
Sector_Economic    0
Tram_Horari        0
Valor              0
dtype: int64

Cambio el formato de `Data` a `datetime`

In [6]:
df_consum_2022['Data']= pd.to_datetime(df_consum_2022['Data'], format='%Y-%m-%d')

La columna de `Valor`la cambio de formato a `float`

In [7]:
df_consum_2022["Valor"] = df_consum_2022["Valor"].astype(float)

Observo que existe una fila en los registros con el valor "No consta" dentro de los tramos horarios, aprovecharé las filas de este registro para conste el consumo total del sector correspondiente durante el día. Aplicaré la suma de los 4 registros anteriores en la columna porque cada barrio/sector contiene 4 tramos horarios durante el día.

In [8]:
# Identifico las filas que incluyen el "No consta"
mask = df_consum_2022["Tram_Horari"] == "No consta"

# Aplico la suma de los 4 registros anteriores en la columna Valor
df_consum_2022.loc[mask, "Valor"] = df_consum_2022["Valor"].shift(1).rolling(window=4, min_periods=1).sum()

# Reemplazo "No consta" por "De 00:00:00 a 23:59:59 h" en Tram_Horari para que conste el consumo de todo el día
df_consum_2022.loc[mask, "Tram_Horari"] = "De 00:00:00 a 23:59:59 h"

# Compruebo la aplicación del tramo modificado
df_consum_2022.head(10)

,Any,Data,Codi_Postal,Sector_Economic,Tram_Horari,Valor
0,2022,2022-01-01,8001,Indústria,De 00:00:00 a 05:59:59 h,817.0
1,2022,2022-01-01,8001,Indústria,De 06:00:00 a 11:59:59 h,942.0
2,2022,2022-01-01,8001,Indústria,De 12:00:00 a 17:59:59 h,1145.0
3,2022,2022-01-01,8001,Indústria,De 18:00:00 a 23:59:59 h,1184.0
4,2022,2022-01-01,8001,Indústria,De 00:00:00 a 23:59:59 h,4088.0
5,2022,2022-01-01,8001,Residencial,De 00:00:00 a 05:59:59 h,24464.0
6,2022,2022-01-01,8001,Residencial,De 06:00:00 a 11:59:59 h,24860.0
7,2022,2022-01-01,8001,Residencial,De 12:00:00 a 17:59:59 h,36160.0
8,2022,2022-01-01,8001,Residencial,De 18:00:00 a 23:59:59 h,38349.0
9,2022,2022-01-01,8001,Residencial,De 00:00:00 a 23:59:59 h,123833.0


Observo que los códigos postales no incluyen el cero inicial, los códigos postales constan de 5 dígitos y considero importante mantener ese formato, agrego el cero dentro de los registros de esa columna

In [9]:
# Agrego un cero al inicio de los valores de la columna "Codi_Postal"
df_consum_2022['Codi_Postal'] = '0' + df_consum_2022['Codi_Postal'].astype(str)

df_consum_2022.head()

,Any,Data,Codi_Postal,Sector_Economic,Tram_Horari,Valor
0,2022,2022-01-01,08001,Indústria,De 00:00:00 a 05:59:59 h,817.0
1,2022,2022-01-01,08001,Indústria,De 06:00:00 a 11:59:59 h,942.0
2,2022,2022-01-01,08001,Indústria,De 12:00:00 a 17:59:59 h,1145.0
3,2022,2022-01-01,08001,Indústria,De 18:00:00 a 23:59:59 h,1184.0
4,2022,2022-01-01,08001,Indústria,De 00:00:00 a 23:59:59 h,4088.0


Para evitar problemas al momento de importar los datos, en la columna `Sector_Economic` aparece la palabra `Indústria` con acento, lo elimino para evitar problemas con la codificación, en las visualizaciones esto puede cambiarse.

In [10]:
df_consum_2022['Sector_Economic'] = df_consum_2022['Sector_Economic'].replace('Indústria', 'Industria')

df_consum_2022.head()

,Any,Data,Codi_Postal,Sector_Economic,Tram_Horari,Valor
0,2022,2022-01-01,08001,Industria,De 00:00:00 a 05:59:59 h,817.0
1,2022,2022-01-01,08001,Industria,De 06:00:00 a 11:59:59 h,942.0
2,2022,2022-01-01,08001,Industria,De 12:00:00 a 17:59:59 h,1145.0
3,2022,2022-01-01,08001,Industria,De 18:00:00 a 23:59:59 h,1184.0
4,2022,2022-01-01,08001,Industria,De 00:00:00 a 23:59:59 h,4088.0


Elimino la columna `Any` al considerarla redundante.

In [11]:
df_consum_2022= df_consum_2022.drop(columns= ['Any'])

Al contener datos de distintas fuentes oficiales opto por unificar los nombres de las columnas en castellano.

In [12]:
df_consum_2022= df_consum_2022.set_axis(['fecha', 'codigo_postal', 'sector_economico', 'tramo_horario', 'consumo'], axis='columns')
df_consum_2022.head()

,fecha,codigo_postal,sector_economico,tramo_horario,consumo
0,2022-01-01,08001,Industria,De 00:00:00 a 05:59:59 h,817.0
1,2022-01-01,08001,Industria,De 06:00:00 a 11:59:59 h,942.0
2,2022-01-01,08001,Industria,De 12:00:00 a 17:59:59 h,1145.0
3,2022-01-01,08001,Industria,De 18:00:00 a 23:59:59 h,1184.0
4,2022-01-01,08001,Industria,De 00:00:00 a 23:59:59 h,4088.0


Para poder identificar fácilmente los tramos horarios agrego una columna para que estos tengan el nombre que les correspondería durante el dia.

In [ ]:
# Defino el diccionario de mapeo
mapeo_tramo = {
    "De 00:00:00 a 05:59:59 h": "Madrugada",
    "De 06:00:00 a 11:59:59 h": "Mañana",
    "De 12:00:00 a 17:59:59 h": "Tarde",
    "De 18:00:00 a 23:59:59 h": "Noche",
    "De 00:00:00 a 23:59:59 h": "Dia"
}

# Creo la nueva columna "tramo_dia" usando el mapeo
df_consum_2022['tramo_dia'] = df_consum_2022['tramo_horario'].map(mapeo_tramo)

Agrego otra columna para identificar a los barrios que corresponden a cada código postal.

In [14]:
mapeo_codigos= {
    "08001": "Ciutat Vella-El Raval",
    "08002": "Barri Gotic",
    "08003": "Barceloneta-Sant Pere",
    "08004": "Poble Sec",
    "08005": "Poble Nou",
    "08006": "Sant Gervasi",
    "08007": "Passeig de Gracia-Rambla de Catalunya",
    "08008": "Diagonal-Provença",
    "08009": "Dreta de L Eixample",
    "08010": "Tetuan-Urquinaona",
    "08011": "Urgell-Sant Antoni",
    "08012": "Vila de Gracia",
    "08013": "Glories Catalanes",
    "08014": "Hostafrancs-Sants",
    "08015": "Sant Antoni",
    "08016": "La Prosperitat-Porta",
    "08017": "Vallvidrera-Tibidabo i Les Planes",
    "08018": "Sant Marti",
    "08019": "Diagonal Mar i El Front Maritim Del Poblenou",
    "08020": "La Verneda i La Pau",
    "08021": "Muntaner",
    "08022": "Sant Gervasi-La Bonanova",
    "08023": "Vallcarca i Los Penitentes",
    "08024": "Gracia-La Salut",
    "08025": "Sagrada Familia",
    "08026": "El Clot",
    "08027": "La Sagrera",
    "08028":  "Les Corts",
    "08029": "L Nou Esquerra de L Eixample",
    "08030": "Sant Andreu-Bon Pastor",
    "08031": "Vilapicina i La Torre Llobeta",
    "08032": "El Carmel-El Guinardo",
    "08033": "Vallbona-Ciutat Meridiana-Torre Baro-Canyelles",
    "08034": "Pedralbes-Sarria",
    "08035": "Sant Genis Dels Agudells-Vall D Hebron-Horta",
    "08036": "L Antiga Esquerra de L Eixample",
    "08037": "Verdaguer-L Eixample",
    "08038": "Sants- Montjuic-La Marina de Port",
    "08039": "Port Vell-Port de Barcelona",
    "08040": "Zona Franca - Port",
    "08041": "Guinardo i Hospital de Sant Pau",
    "08042": "La Guineueta-Verdun"
}

df_consum_2022['barrio']= df_consum_2022['codigo_postal'].map(mapeo_codigos)

df_consum_2022.head()

,fecha,codigo_postal,sector_economico,tramo_horario,consumo,tramo_dia,barrio
0,2022-01-01,08001,Industria,De 00:00:00 a 05:59:59 h,817.0,Madrugada,Ciutat Vella-El Raval
1,2022-01-01,08001,Industria,De 06:00:00 a 11:59:59 h,942.0,Mañana,Ciutat Vella-El Raval
2,2022-01-01,08001,Industria,De 12:00:00 a 17:59:59 h,1145.0,Tarde,Ciutat Vella-El Raval
3,2022-01-01,08001,Industria,De 18:00:00 a 23:59:59 h,1184.0,Noche,Ciutat Vella-El Raval
4,2022-01-01,08001,Industria,De 00:00:00 a 23:59:59 h,4088.0,Dia,Ciutat Vella-El Raval


Reordeno las columnas.

In [15]:
df_consum_2022= df_consum_2022[['fecha', 'codigo_postal', 'barrio', 'sector_economico', 'tramo_horario', 'tramo_dia', 'consumo']]

df_consum_2022.head(10)

,fecha,codigo_postal,barrio,sector_economico,tramo_horario,tramo_dia,consumo
0,2022-01-01,08001,Ciutat Vella-El Raval,Industria,De 00:00:00 a 05:59:59 h,Madrugada,817.0
1,2022-01-01,08001,Ciutat Vella-El Raval,Industria,De 06:00:00 a 11:59:59 h,Mañana,942.0
2,2022-01-01,08001,Ciutat Vella-El Raval,Industria,De 12:00:00 a 17:59:59 h,Tarde,1145.0
3,2022-01-01,08001,Ciutat Vella-El Raval,Industria,De 18:00:00 a 23:59:59 h,Noche,1184.0
4,2022-01-01,08001,Ciutat Vella-El Raval,Industria,De 00:00:00 a 23:59:59 h,Dia,4088.0
5,2022-01-01,08001,Ciutat Vella-El Raval,Residencial,De 00:00:00 a 05:59:59 h,Madrugada,24464.0
6,2022-01-01,08001,Ciutat Vella-El Raval,Residencial,De 06:00:00 a 11:59:59 h,Mañana,24860.0
7,2022-01-01,08001,Ciutat Vella-El Raval,Residencial,De 12:00:00 a 17:59:59 h,Tarde,36160.0
8,2022-01-01,08001,Ciutat Vella-El Raval,Residencial,De 18:00:00 a 23:59:59 h,Noche,38349.0
9,2022-01-01,08001,Ciutat Vella-El Raval,Residencial,De 00:00:00 a 23:59:59 h,Dia,123833.0


Guardo el dataframe en formato csv

In [ ]:
df_consum_2022.to_csv('consumo_2023.csv', index= False)